# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 11:44AM,240620,15,8199624,"Laser Pharmaceuticals, LLC",Released
1,Jun 28 2022 11:44AM,240620,15,8201019,"Laser Pharmaceuticals, LLC",Released
2,Jun 28 2022 11:44AM,240620,15,8201020,"Laser Pharmaceuticals, LLC",Released
3,Jun 28 2022 11:44AM,240620,15,8201021,"Laser Pharmaceuticals, LLC",Released
4,Jun 28 2022 11:44AM,240620,15,8201022,"Laser Pharmaceuticals, LLC",Released
5,Jun 28 2022 11:44AM,240620,15,8201023,"Laser Pharmaceuticals, LLC",Released
6,Jun 28 2022 11:44AM,240620,15,8201024,"Laser Pharmaceuticals, LLC",Released
7,Jun 28 2022 11:44AM,240620,15,8204193,"Laser Pharmaceuticals, LLC",Released
8,Jun 28 2022 11:44AM,240620,15,8204194,"Laser Pharmaceuticals, LLC",Released
9,Jun 28 2022 11:44AM,240620,15,8204195,"Laser Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,240617,Released,1
37,240618,Released,1
38,240619,Released,1
39,240620,Released,19
40,240621,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240617,NaN,NaN,1.0
240618,NaN,NaN,1.0
240619,NaN,NaN,1.0
240620,NaN,NaN,19.0
240621,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240534,0.0,0.0,1.0
240535,0.0,1.0,0.0
240549,0.0,0.0,1.0
240550,0.0,0.0,1.0
240574,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240534,0,0,1
240535,0,1,0
240549,0,0,1
240550,0,0,1
240574,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240534,0,0,1
1,240535,0,1,0
2,240549,0,0,1
3,240550,0,0,1
4,240574,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240534,,,1
1,240535,,1,
2,240549,,,1
3,240550,,,1
4,240574,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC"
19,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC"
20,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.
21,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC"
22,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC"
23,Jun 28 2022 11:19AM,240617,22,"NBTY Global, Inc."
24,Jun 28 2022 11:18AM,240608,15,"Carwin Pharmaceutical Associates, LLC"
31,Jun 28 2022 11:17AM,240609,15,"Canton Laboratories, LLC"
32,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc."
33,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC",,,19
1,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC",,,1
2,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.,,,1
3,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC",,,1
4,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC",,,1
5,Jun 28 2022 11:19AM,240617,22,"NBTY Global, Inc.",,,1
6,Jun 28 2022 11:18AM,240608,15,"Carwin Pharmaceutical Associates, LLC",,,7
7,Jun 28 2022 11:17AM,240609,15,"Canton Laboratories, LLC",,,1
8,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",,,1
9,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC",19,,
1,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC",1,,
2,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.,1,,
3,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC",1,,
4,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC",1,,
5,Jun 28 2022 11:19AM,240617,22,"NBTY Global, Inc.",1,,
6,Jun 28 2022 11:18AM,240608,15,"Carwin Pharmaceutical Associates, LLC",7,,
7,Jun 28 2022 11:17AM,240609,15,"Canton Laboratories, LLC",1,,
8,Jun 28 2022 11:10AM,240616,19,"GUSA Granules USA, Inc.",1,,
9,Jun 28 2022 11:09AM,240615,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC",19,,
1,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC",1,,
2,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.,1,,
3,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC",1,,
4,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC",19.0,NaN,NaN
1,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC",1.0,NaN,NaN
2,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC",1.0,NaN,NaN
4,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 11:44AM,240620,15,"Laser Pharmaceuticals, LLC",19.0,0.0,0.0
1,Jun 28 2022 11:44AM,240619,15,"Mizner Bioscience, LLC",1.0,0.0,0.0
2,Jun 28 2022 11:43AM,240596,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Jun 28 2022 11:35AM,240621,19,"NAPP Technologies, LLC",1.0,0.0,0.0
4,Jun 28 2022 11:22AM,240618,19,"Graystone, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1684194,20.0,10.0,0.0
15,1443669,81.0,8.0,0.0
16,481187,4.0,7.0,0.0
17,240550,1.0,0.0,0.0
18,240614,1.0,0.0,0.0
19,2405962,36.0,20.0,7.0
20,240599,15.0,6.0,0.0
21,962222,3.0,1.0,0.0
22,240617,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1684194,20.0,10.0,0.0
1,15,1443669,81.0,8.0,0.0
2,16,481187,4.0,7.0,0.0
3,17,240550,1.0,0.0,0.0
4,18,240614,1.0,0.0,0.0
5,19,2405962,36.0,20.0,7.0
6,20,240599,15.0,6.0,0.0
7,21,962222,3.0,1.0,0.0
8,22,240617,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,20.0,10.0,0.0
1,15,81.0,8.0,0.0
2,16,4.0,7.0,0.0
3,17,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,36.0,20.0,7.0
6,20,15.0,6.0,0.0
7,21,3.0,1.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,20.0
1,15,Released,81.0
2,16,Released,4.0
3,17,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
Executing,10.0,8.0,7.0,0.0,0.0,20.0,6.0,1.0,0.0
Released,20.0,81.0,4.0,1.0,1.0,36.0,15.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
1,Executing,10.0,8.0,7.0,0.0,0.0,20.0,6.0,1.0,0.0
2,Released,20.0,81.0,4.0,1.0,1.0,36.0,15.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0
1,Executing,10.0,8.0,7.0,0.0,0.0,20.0,6.0,1.0,0.0
2,Released,20.0,81.0,4.0,1.0,1.0,36.0,15.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()